In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

# Replace 'your-folder-path' with the path to your folder in Google Drive
folder_path = '/content/drive/MyDrive/Capstone/Data/Version1/images'

# List the files to verify access
os.listdir(folder_path)


['frame_666.jpg',
 'frame_1257.jpg',
 'frame_714.jpg',
 'frame_910.jpg',
 'frame_1314.jpg',
 'frame_1055.jpg',
 'frame_1318.jpg',
 'frame_920.jpg',
 'frame_916.jpg',
 'frame_906.jpg',
 'frame_1342.jpg',
 'frame_510.jpg',
 'frame_1308.jpg',
 'frame_1256.jpg',
 'frame_878.jpg',
 'frame_1288.jpg',
 'frame_742.jpg',
 'frame_1251.jpg',
 'frame_708.jpg',
 'frame_693.jpg',
 'frame_1083.jpg',
 'frame_1464.jpg',
 'frame_1131.jpg',
 'frame_676.jpg',
 'frame_1294.jpg',
 'frame_852.jpg',
 'frame_864.jpg',
 'frame_1086.jpg',
 'frame_670.jpg',
 'frame_874.jpg',
 'frame_862.jpg',
 'frame_1141.jpg',
 'frame_1478.jpg',
 'frame_1137.jpg',
 'frame_516.jpg',
 'frame_1090.jpg',
 'frame_1332.jpg',
 'frame_1304.jpg',
 'frame_1096.jpg',
 'frame_703.jpg',
 'frame_868.jpg',
 'frame_1250.jpg',
 'frame_1049.jpg',
 'frame_733 (1).jpg',
 'frame_1469.jpg',
 'frame_1315.jpg',
 'frame_1044.jpg',
 'frame_1299.jpg',
 'frame_1474.jpg',
 'frame_1087.jpg',
 'frame_1126.jpg',
 'frame_732 (1).jpg',
 'frame_506.jpg',
 'frame_

In [ ]:
import os

# Replace 'your-folder-path' with the path to your folder in Google Drive
folder_path = '/content/drive/MyDrive/Capstone/Data/Version1/annotations'

# List the files to verify access
os.listdir(folder_path)


['frame_516.xml',
 'frame_1090.xml',
 'frame_670.xml',
 'frame_868.xml',
 'frame_1055.xml',
 'frame_874.xml',
 'frame_864.xml',
 'frame_1452.xml',
 'frame_703.xml',
 'frame_1141.xml',
 'frame_1332.xml',
 'frame_676.xml',
 'frame_1049.xml',
 'frame_1257.xml',
 'frame_1318.xml',
 'frame_510.xml',
 'frame_708.xml',
 'frame_1294.xml',
 'frame_714.xml',
 'frame_1127.xml',
 'frame_1131.xml',
 'frame_1465.xml',
 'frame_1137.xml',
 'frame_1083.xml',
 'frame_1251.xml',
 'frame_1288.xml',
 'frame_862.xml',
 'frame_743.xml',
 'frame_1304.xml',
 'frame_920.xml',
 'frame_1096.xml',
 'frame_852.xml',
 'frame_1479.xml',
 'frame_742.xml',
 'frame_1314.xml',
 'frame_906.xml',
 'frame_1478.xml',
 'frame_910.xml',
 'frame_1469.xml',
 'frame_506.xml',
 'frame_1342.xml',
 'frame_1260.xml',
 'frame_916.xml',
 'frame_1261.xml',
 'frame_521.xml',
 'frame_1100.xml',
 'frame_1298.xml',
 'frame_693.xml',
 'frame_1073.xml',
 'frame_507.xml',
 'frame_1059.xml',
 'frame_1101.xml',
 'frame_1087.xml',
 'frame_704.xml

In [ ]:
# !pip install opencv-python lxml


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import cv2
import xml.etree.ElementTree as ET

# Define paths
images_folder_path = '/content/drive/MyDrive/Capstone/Data/Version1/images'
annotations_folder_path = '/content/drive/MyDrive/Capstone/Data/Version1/annotations'
output_images_folder_path = '/content/drive/MyDrive/Capstone/Data/Version1/resized_images'
output_annotations_folder_path = '/content/drive/MyDrive/Capstone/Data/Version1/resized_annotations'

# Create output directories if they don't exist
os.makedirs(output_images_folder_path, exist_ok=True)
os.makedirs(output_annotations_folder_path, exist_ok=True)

# Target size
target_size = (224, 224)

# Function to resize image and adjust XML
def resize_image_and_adjust_xml(image_path, xml_path, output_image_path, output_xml_path):
    # Load image
    image = cv2.imread(image_path)
    original_height, original_width = image.shape[:2]

    # Resize image
    resized_image = cv2.resize(image, target_size)
    cv2.imwrite(output_image_path, resized_image)

    # Adjust XML
    tree = ET.parse(xml_path)
    root = tree.getroot()

    for size in root.findall('size'):
        size.find('width').text = str(target_size[0])
        size.find('height').text = str(target_size[1])

    for obj in root.findall('object'):
        xml_bndbox = obj.find('bndbox')
        xmin = int(xml_bndbox.find('xmin').text)
        ymin = int(xml_bndbox.find('ymin').text)
        xmax = int(xml_bndbox.find('xmax').text)
        ymax = int(xml_bndbox.find('ymax').text)

        # Scale bounding box
        xmin = int(xmin * target_size[0] / original_width)
        xmax = int(xmax * target_size[0] / original_width)
        ymin = int(ymin * target_size[1] / original_height)
        ymax = int(ymax * target_size[1] / original_height)

        xml_bndbox.find('xmin').text = str(xmin)
        xml_bndbox.find('ymin').text = str(ymin)
        xml_bndbox.find('xmax').text = str(xmax)
        xml_bndbox.find('ymax').text = str(ymax)

    # Write updated XML to file
    tree.write(output_xml_path)

# Process each image and corresponding XML
for filename in os.listdir(images_folder_path):
    if filename.endswith(".jpg"):  # Assuming images are in JPG format
        image_path = os.path.join(images_folder_path, filename)
        xml_filename = filename.replace(".jpg", ".xml")
        xml_path = os.path.join(annotations_folder_path, xml_filename)

        output_image_path = os.path.join(output_images_folder_path, filename)
        output_xml_path = os.path.join(output_annotations_folder_path, xml_filename)

        if os.path.exists(xml_path):  # Ensure there is a corresponding XML file
            resize_image_and_adjust_xml(image_path, xml_path, output_image_path, output_xml_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import cv2
import numpy as np

# Paths to the resized images and the output directory for normalized images
resized_images_folder_path = '/content/drive/MyDrive/Capstone/Data/Version1/resized_images'
output_normalized_images_folder_path = '/content/drive/MyDrive/Capstone/Data/Version1/normalized_images'

# Create output directory if it doesn't exist
os.makedirs(output_normalized_images_folder_path, exist_ok=True)

# Function to normalize the pixel values of an image
def normalize_image(image_path, output_image_path):
    # Load image
    image = cv2.imread(image_path)

    # Normalize the pixel values to the range [0, 1]
    normalized_image = image / 255.0

    # Convert the normalized image to float32 format
    normalized_image = normalized_image.astype(np.float32)

    # Save the normalized image
    # We save it as a .png to preserve the float32 format.
    cv2.imwrite(output_image_path, normalized_image * 255)

# Process each resized image
for filename in os.listdir(resized_images_folder_path):
    if filename.endswith(".jpg"):  # Assuming images are in JPG format
        image_path = os.path.join(resized_images_folder_path, filename)
        output_image_path = os.path.join(output_normalized_images_folder_path, filename)

        # Normalize the image and save it
        normalize_image(image_path, output_image_path)
